want: 
- reproduce visualization from pptk tutorial
- get summary statistics 
    - number of users 
	- median/quartiles of number of locations visited 

In [19]:
# import packages and dependencies
import pandas as pd
import matplotlib.pyplot as plt
import folium
from PIL import Image, ImageDraw
import sys
sys.path.insert(1, '../src')

import read_geolife

In [1]:


# testing reading data and assigning labels  for a single user 
df_all_users = read_geolife.read_all_users('../data/')


CHANGE
[1/182] processing user 135
CHANGE
[2/182] processing user 132
CHANGE
[3/182] processing user 104


../src/read_geolife.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  points['label'][no_label] = 0


CHANGE
[4/182] processing user 103
CHANGE
[5/182] processing user 168
CHANGE
[6/182] processing user 157
CHANGE
[7/182] processing user 150
CHANGE
[8/182] processing user 159
CHANGE
[9/182] processing user 166
CHANGE
[10/182] processing user 161
CHANGE
[11/182] processing user 102
CHANGE
[12/182] processing user 105
CHANGE
[13/182] processing user 133
CHANGE
[14/182] processing user 134
CHANGE
[15/182] processing user 160
CHANGE
[16/182] processing user 158
CHANGE
[17/182] processing user 167
CHANGE
[18/182] processing user 151
CHANGE
[19/182] processing user 169
CHANGE
[20/182] processing user 156
CHANGE
[21/182] processing user 024
CHANGE
[22/182] processing user 023
CHANGE
[23/182] processing user 015
CHANGE
[24/182] processing user 012
CHANGE
[25/182] processing user 079
CHANGE
[26/182] processing user 046
CHANGE
[27/182] processing user 041
CHANGE
[28/182] processing user 048
CHANGE
[29/182] processing user 077
CHANGE
[30/182] processing user 083
CHANGE
[31/182] processing user 08

In [12]:
df_all_users.to_pickle('../results/geolife.pkl')
df = pd.read_pickle('../results/geolife.pkl')

In [13]:
df[['lat', 'lon', 'alt']].describe()

,lat,lon,alt
count,2.487698e+07,2.487698e+07,2.487698e+07
mean,3.897989e+01,1.123597e+02,5.130172e+02
std,4.070501e+00,2.717960e+01,2.301976e+03
min,1.044024e+00,-1.799696e+02,-3.264760e+04
25%,3.990636e+01,1.163065e+02,7.870000e+01
50%,3.997498e+01,1.163342e+02,1.476000e+02
75%,3.999848e+01,1.164194e+02,2.190000e+02
max,4.001667e+02,1.799969e+02,1.075033e+05


2024-02-20-tues
TODO: 
- clean up notebook loll 
- continue cross referencing with the data documentation + keep filtering to deal with nonsensical values
- discretize the surface covered by these gps coordinates (e.g. google s2 or something else) such that each element of that partition corresponds to a “location”
- visualize trajectories some random subset of the users — do you know good Python packages for visualizing gps data? I was thinking Folium and GeoPandas — also this tutorial seems promising: https://courses.spatialthoughts.com/python-dataviz.html

https://geopandas.org/en/stable/docs/user_guide/mapping.html
https://python-visualization.github.io/folium/latest/getting_started.html

Thanks for the suggestions! Looks like folium and geopandas are the way to go w/r/t visualization (and partitioning locations). I’ll check these out once I’m done cleaning the data and plot some sample trajectories. I’ll also generate some user-level summaries.